In [4]:
import pandas as pd

In [3]:
!nvidia-smi

Sun Dec 17 09:23:25 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [5]:
!pip install transformers

In [6]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
import shutil
import sys

In [7]:
#Dataset path
train_path = "/content/drive/MyDrive/hyprsol/data.csv"

In [8]:
#Reading into dataFrame
train_df = pd.read_csv(train_path, sep='\t')

In [9]:
train_df['description'].values

array(['Develop a predictive analysis software that uses existing data to forecast trends, patterns, and behaviours.',
       'Implement a system for managing employee work hours, schedule changes, and leave requests.',
       'A virtual platform that contains an extensive collection of books, articles, and journals.',
       'Software to maintain and manage the medical records and information of patients in a hospital.',
       'A platform that provides on-demand streaming of TV shows and movie content to users.',
       'An online learning hub that facilitates virtual interactions between teachers and students.',
       'A tool designed to assist teams in organizing work and managing projects and tasks effectively.',
       'A system used to automate basic home functions like temperature control, door locking, etc using IoT.',
       'A streamlined e-commerce website. Focused on providing an organized interface for customers to purchase goods with ease. Will require catalog browsing,

In [10]:
#combining column with textual informations
train_df['CONTEXT'] = train_df['name'] + ". " + train_df['description']

In [11]:
#dropping the unwanted colmns
train_df.drop(labels=['Unnamed: 0','name', 'description'], axis=1, inplace=True)

In [12]:
train_df.columns.tolist()[0:-1]

['Analytics',
 'Archiving and Digital Preservation (DP)',
 'Automation',
 'Backup',
 'Blogging Platforms',
 'Booking and Scheduling',
 'Bookmarks and Link Sharing',
 'Calendar & Contacts',
 'Communication - Custom Communication Systems',
 'Communication - Email - Complete Solutions',
 'Communication - Email - Mail Delivery Agents',
 'Communication - Email - Mail Transfer Agents',
 'Communication - Email - Mailing Lists and Newsletters',
 'Communication - Email - Webmail Clients',
 'Communication - IRC',
 'Communication - SIP',
 'Communication - Social Networks and Forums',
 'Communication - Video Conferencing',
 'Communication - XMPP - Servers',
 'Communication - XMPP - Web Clients',
 'Community-Supported Agriculture (CSA)',
 'Conference Management',
 'Content Management Systems (CMS)',
 'Database Management',
 'DNS',
 'Document Management - E-books',
 'Document Management - Institutional Repository and Digital Library Software',
 'Document Management - Integrated Library Systems (ILS)

In [13]:
# rearranging columns
train_df = train_df[['CONTEXT'] + train_df.columns.tolist()[0:-1] ]

In [14]:
train_df.head()

,CONTEXT,Analytics,Archiving and Digital Preservation (DP),Automation,Backup,Blogging Platforms,Booking and Scheduling,Bookmarks and Link Sharing,Calendar & Contacts,Communication - Custom Communication Systems,...,Static Site Generators,Status / Uptime pages,Task Management & To-do Lists,Ticketing,Time Tracking,URL Shorteners,Video Surveillance,VPN,Web Servers,Wikis
0,Predictive Analysis Software. Develop a predic...,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Employee Scheduling System. Implement a system...,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Digital Library. A virtual platform that conta...,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Healthcare Management System. Software to main...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Video Streaming Service. A platform that provi...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
#Initializing the target array for the model
target_list = train_df.columns.tolist()[1:]

In [16]:
target_list

['Analytics',
 'Archiving and Digital Preservation (DP)',
 'Automation',
 'Backup',
 'Blogging Platforms',
 'Booking and Scheduling',
 'Bookmarks and Link Sharing',
 'Calendar & Contacts',
 'Communication - Custom Communication Systems',
 'Communication - Email - Complete Solutions',
 'Communication - Email - Mail Delivery Agents',
 'Communication - Email - Mail Transfer Agents',
 'Communication - Email - Mailing Lists and Newsletters',
 'Communication - Email - Webmail Clients',
 'Communication - IRC',
 'Communication - SIP',
 'Communication - Social Networks and Forums',
 'Communication - Video Conferencing',
 'Communication - XMPP - Servers',
 'Communication - XMPP - Web Clients',
 'Community-Supported Agriculture (CSA)',
 'Conference Management',
 'Content Management Systems (CMS)',
 'Database Management',
 'DNS',
 'Document Management - E-books',
 'Document Management - Institutional Repository and Digital Library Software',
 'Document Management - Integrated Library Systems (ILS)

In [17]:
train_df[target_list].values

array([[1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [18]:
#Initializing the parameters for training
MAX_LEN = 256
TRAIN_BATCH_SIZE = 32
VALID_BATCH_SIZE = 32
EPOCHS = 200
LEARNING_RATE = 1e-05# hy

In [19]:
from transformers import BertTokenizer, BertModel

In [20]:
#Initializing the tokenizer for preprocessing
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [21]:
#A custom dataset class in which we can create a dataset object with torch's dataset attributes
class CustomDataset(torch.utils.data.Dataset):

    def __init__(self, df, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.df = df
        self.title = df['CONTEXT']
        self.targets = self.df[target_list].values
        self.max_len = max_len

    def __len__(self):
        return len(self.title)

    def __getitem__(self, index):
        title = str(self.title[index])
        title = " ".join(title.split())

        inputs = self.tokenizer.encode_plus(
            title,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            return_token_type_ids=True,
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )

        return {
            'input_ids': inputs['input_ids'].flatten(),
            'attention_mask': inputs['attention_mask'].flatten(),
            'token_type_ids': inputs["token_type_ids"].flatten(),
            'targets': torch.FloatTensor(self.targets[index])
        }

In [22]:
#Splitting the dataset into training and validation
train_size = 0.8
temp = train_df
train_df = train_df.sample(frac=train_size, random_state=200).reset_index(drop=True)
val_df = temp.drop(train_df.index).reset_index(drop=True)

In [23]:
#Creating our custom dataset
train_dataset = CustomDataset(train_df, tokenizer, MAX_LEN)
valid_dataset = CustomDataset(val_df, tokenizer, MAX_LEN)

In [24]:
#Creating the dataset loaders
train_data_loader = torch.utils.data.DataLoader(train_dataset,
    batch_size=TRAIN_BATCH_SIZE,
    shuffle=True,
    num_workers=0
)

val_data_loader = torch.utils.data.DataLoader(valid_dataset,
    batch_size=VALID_BATCH_SIZE,
    shuffle=False,
    num_workers=0
)

In [25]:
len(val_data_loader)

6

In [26]:
for data in train_data_loader:
  print(data)

{'input_ids': tensor([[  101,  8702,  7905,  ...,     0,     0,     0],
        [  101,  2833,  6959,  ...,     0,     0,     0],
        [  101,  2665,  7639,  ...,     0,     0,     0],
        ...,
        [  101,  2951,  8822,  ...,     0,     0,     0],
        [  101,  3784,  2607,  ...,     0,     0,     0],
        [  101,  6556, 14924,  ...,     0,     0,     0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'targets': tensor([[0., 1., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
     

In [27]:
#Loading torch device CUDA if support available
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [27]:
device

device(type='cuda')

In [28]:
def load_ckp(checkpoint_fpath, model, optimizer):
    """
    checkpoint_path: path to save checkpoint
    model: model that we want to load checkpoint parameters into
    optimizer: optimizer we defined in previous training
    """
    # load check point
    checkpoint = torch.load(checkpoint_fpath)
    # initialize state_dict from checkpoint to model
    model.load_state_dict(checkpoint['state_dict'])
    # initialize optimizer from checkpoint to optimizer
    optimizer.load_state_dict(checkpoint['optimizer'])
    # initialize valid_loss_min from checkpoint to valid_loss_min
    valid_loss_min = checkpoint['valid_loss_min']
    # return model, optimizer, epoch value, min validation loss
    return model, optimizer, checkpoint['epoch'], valid_loss_min.item()

def save_ckp(state, is_best, checkpoint_path, best_model_path):
    """
    state: checkpoint we want to save
    is_best: is this the best checkpoint; min validation loss
    checkpoint_path: path to save checkpoint
    best_model_path: path to save best model
    """
    f_path = checkpoint_path
    # save checkpoint data to the path given, checkpoint_path
    torch.save(state, f_path)
    # if it is a best model, min validation loss
    if is_best:
        best_fpath = best_model_path
        # copy that checkpoint file to best path given, best_model_path
        shutil.copyfile(f_path, best_fpath)

In [47]:
class BERTClass(torch.nn.Module):
    def __init__(self):
        super(BERTClass, self).__init__()
        self.bert_model = BertModel.from_pretrained('bert-base-uncased', return_dict=True)
        self.dropout = torch.nn.Dropout(0.3)
        self.linear = torch.nn.Linear(768, 88)

    def forward(self, input_ids, attn_mask, token_type_ids):
        output = self.bert_model(
            input_ids,
            attention_mask=attn_mask,
            token_type_ids=token_type_ids
        )
        output_dropout = self.dropout(output.pooler_output)
        output = self.linear(output_dropout)
        return output

model = BERTClass()
model.to(device)

BERTClass(
  (bert_model): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_a

In [30]:
def loss_fn(outputs, targets):
    return torch.nn.BCEWithLogitsLoss()(outputs, targets)

optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

In [31]:
val_targets=[]
val_outputs=[]

In [48]:
def train_model(n_epochs, training_loader, validation_loader, model,
                optimizer, checkpoint_path, best_model_path):

  # initialize tracker for minimum validation loss
  valid_loss_min = np.Inf


  for epoch in range(1, n_epochs+1):
    train_loss = 0
    valid_loss = 0

    model.train()
    print('############# Epoch {}: Training Start   #############'.format(epoch))
    for batch_idx, data in enumerate(training_loader):
        ids = data['input_ids'].to(device, dtype = torch.long)
        mask = data['attention_mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.float)

        outputs = model(ids, mask, token_type_ids)

        optimizer.zero_grad()
        loss = loss_fn(outputs, targets)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        #print('before loss data in training', loss.item(), train_loss)
        train_loss = train_loss + ((1 / (batch_idx + 1)) * (loss.item() - train_loss))
        #print('after loss data in training', loss.item(), train_loss)

    print('############# Epoch {}: Training End     #############'.format(epoch))

    print('############# Epoch {}: Validation Start   #############'.format(epoch))

    #Validation process
    model.eval()

    with torch.no_grad():
      for batch_idx, data in enumerate(validation_loader, 0):
            ids = data['input_ids'].to(device, dtype = torch.long)
            mask = data['attention_mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
            targets = data['targets'].to(device, dtype = torch.float)
            outputs = model(ids, mask, token_type_ids)

            loss = loss_fn(outputs, targets)
            valid_loss = valid_loss + ((1 / (batch_idx + 1)) * (loss.item() - valid_loss))
            val_targets.extend(targets.cpu().detach().numpy().tolist())
            val_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())

      print('############# Epoch {}: Validation End     #############'.format(epoch))
      # calculate average losses
      #print('before cal avg train loss', train_loss)
      train_loss = train_loss/len(training_loader)
      valid_loss = valid_loss/len(validation_loader)
      # print training/validation statistics
      print('Epoch: {} \tAvgerage Training Loss: {:.6f} \tAverage Validation Loss: {:.6f}'.format(
            epoch,
            train_loss,
            valid_loss
            ))

      # create checkpoint variable and add important data
      checkpoint = {
            'epoch': epoch + 1,
            'valid_loss_min': valid_loss,
            'state_dict': model.state_dict(),
            'optimizer': optimizer.state_dict()
      }

        # save checkpoint
      save_ckp(checkpoint, False, checkpoint_path, best_model_path)

      if valid_loss <= valid_loss_min:
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(valid_loss_min,valid_loss))
        # save checkpoint as best model
        save_ckp(checkpoint, True, checkpoint_path, best_model_path)
        valid_loss_min = valid_loss

    print('############# Epoch {}  Done   #############\n'.format(epoch))

  return model

In [33]:
#Save paths for the model
ckpt_path = "/content/drive/MyDrive/hyprsol/multi-label/curr_ckpt"
best_model_path = "/content/drive/MyDrive/hyprsol/multi-label/best_model.pt"

In [ ]:
trained_model = train_model(EPOCHS, train_data_loader, val_data_loader, model, optimizer, ckpt_path, best_model_path)

############# Epoch 1: Training Start   #############
############# Epoch 1: Training End     #############
############# Epoch 1: Validation Start   #############
############# Epoch 1: Validation End     #############
Epoch: 1 	Avgerage Training Loss: 0.031782 	Average Validation Loss: 0.100872
Validation loss decreased (inf --> 0.100872).  Saving model ...
############# Epoch 1  Done   #############

############# Epoch 2: Training Start   #############
############# Epoch 2: Training End     #############
############# Epoch 2: Validation Start   #############
############# Epoch 2: Validation End     #############
Epoch: 2 	Avgerage Training Loss: 0.027515 	Average Validation Loss: 0.087191
Validation loss decreased (0.100872 --> 0.087191).  Saving model ...
############# Epoch 2  Done   #############

############# Epoch 3: Training Start   #############
############# Epoch 3: Training End     #############
############# Epoch 3: Validation Start   #############
############# Epo

In [54]:
checkpoint = torch.load(ckpt_path, map_location=torch.device('cpu'),mmap=True)
model.load_state_dict(checkpoint["state_dict"],assign=True)
optimizer.load_state_dict(checkpoint['optimizer'])

In [55]:
# testing
for i, example in enumerate(examples):
  print('Example : ' + str(i))
  encodings = tokenizer.encode_plus(
    example,
    None,
    add_special_tokens=True,
    max_length=MAX_LEN,
    padding='max_length',
    return_token_type_ids=True,
    truncation=True,
    return_attention_mask=True,
    return_tensors='pt'
  )
  model.eval()
  with torch.no_grad():
      input_ids = encodings['input_ids'].to(device, dtype=torch.long)
      attention_mask = encodings['attention_mask'].to(device, dtype=torch.long)
      token_type_ids = encodings['token_type_ids'].to(device, dtype=torch.long)
      output = model(input_ids, attention_mask, token_type_ids)
      final_output = torch.sigmoid(output).cpu().detach().numpy().tolist()
      for output in final_output[0]:
        if (output >= 0.1):
          print(train_df.columns[1:].to_list()[final_output[0].index(output)])
        else:
          continue
  print('~~~~~~~~~~~~~~~~~~~~~~')


Example : 0
Analytics
Communication - Email - Complete Solutions
Content Management Systems (CMS)
Database Management
Human Resources Management (HRM)
Learning and Courses
Software Development - Project Management
Software Development
Time Tracking
~~~~~~~~~~~~~~~~~~~~~~
Example : 1
Automation
Communication - Custom Communication Systems
Internet of Things (IoT)
Monitoring
Software Development - API Management
Software Development
~~~~~~~~~~~~~~~~~~~~~~
Example : 2
Analytics
Database Management
Software Development - Project Management
Software Development
~~~~~~~~~~~~~~~~~~~~~~
Example : 3
Communication - Custom Communication Systems
Communication - Email - Complete Solutions
Communication - Social Networks and Forums
Communication - Video Conferencing
Content Management Systems (CMS)
Learning and Courses
Software Development - Project Management
Software Development
~~~~~~~~~~~~~~~~~~~~~~
Example : 4
Analytics
Communication - Custom Communication Systems
Database Management
Internet 

In [1]:
example1 = ["A fitness application that allows users to track their workouts, nutrition intake, and progress. It also offers personalized workout recommendations based on user goals and fitness level."]
example2 = ["Design and implement a smart home automation system that allows users to control and monitor their home devices remotely. This could include lights, thermostats, security cameras, and more."]
example3 = ["Create a comprehensive health and fitness tracker that helps users monitor and improve their well-being. This can include features for tracking physical activity, nutrition, sleep patterns, and overall health metrics."]
example4 = ["Develop an immersive VR learning platform that provides an engaging and interactive educational experience. This can be tailored to various subjects, from history to science or language learning."]
example5 = ["Create a system that empowers local communities to monitor and report environmental conditions. This can include air and water quality, noise levels, and other relevant factors."]
examples = example1 + example2 + example3 + example4 + example5
examples

['A fitness application that allows users to track their workouts, nutrition intake, and progress. It also offers personalized workout recommendations based on user goals and fitness level.',
 'Design and implement a smart home automation system that allows users to control and monitor their home devices remotely. This could include lights, thermostats, security cameras, and more.',
 'Create a comprehensive health and fitness tracker that helps users monitor and improve their well-being. This can include features for tracking physical activity, nutrition, sleep patterns, and overall health metrics.',
 'Develop an immersive VR learning platform that provides an engaging and interactive educational experience. This can be tailored to various subjects, from history to science or language learning.',
 'Create a system that empowers local communities to monitor and report environmental conditions. This can include air and water quality, noise levels, and other relevant factors.']

In [ ]:
train_df.columns[1:].to_list()[0]